In [1]:
import xarray
import numpy as np
from lme import LME
import lme.utils as utils

#### Read data

In [2]:
n_years = 28
n_locs = 195
n_ages = 23
n_sexes = 2
y_data = xarray.open_dataset('./data/20190726_tb_latent_prev.nc').transpose('location_id', 'age_group_id', 'sex_id', 'year_id')['value'].values
y = y_data[:n_locs, :n_ages, :n_sexes, :n_years].reshape(-1)
Y = np.log(y) - np.log(1-y)
haq_data = xarray.open_dataset('./data/20190726_haq.nc').transpose('location_id', 'age_group_id', 'sex_id', 'year_id')['value'].values
haq = haq_data[:n_locs,:n_ages,:n_sexes,:n_years].reshape(-1)
print(Y.shape, haq.shape)

(251160,) (5460,)


#### Build model

$$ y = \beta \text{HAQ} + \beta_I \text{age-sex} + \pi_l $$

In [3]:
model = LME([n_locs,n_ages,2,n_years], 1, Y, 
            {'haq':(haq - np.mean(haq),[True,False,False,True])},
            indicators={'ind_age-sex':[False,True,True,False]}, 
            global_effects_names=['haq'], 
            global_intercept=False, 
            random_effects={'intercept_location':[True,False,False,False]})

In [4]:
import time
t0 = time.time()
model.optimize(inner_max_iter=200)
print('elapsed', time.time()-t0)

n_groups 195
k_beta 47
k_gamma 1
total number of fixed effects variables 49
elapsed 3.6989309787750244


#### Sample $\beta$ and $\pi$

In [5]:
model.postVarGlobal()
model.postVarRandom()

In [6]:
beta_samples, u_samples = model.draw()

In [7]:
samples_pairs = model.outputDraws(by_type=False)

In [8]:
cov_samples, ind_samples, ran_eff_samples = model.outputDraws(by_type=True, combine_cov=True)

In [9]:
cov_samples

CovDraws(array=array([[-0.02602872, -0.02589877, -0.02585231, -0.02584249, -0.02589662,
        -0.02599817, -0.02607857, -0.02594871, -0.02597201, -0.02605046]]), names=['haq'])

In [10]:
cov_samples, ind_samples, ran_eff_samples = model.outputDraws(by_type=True, combine_cov=False)

In [11]:
cov_samples

[Draws(array=array([-0.02604667, -0.02581138, -0.02585465, -0.02600902, -0.02588871,
        -0.02598752, -0.02590011, -0.02606395, -0.02600862, -0.02591017]), name='haq')]

In [12]:
location_ids = xarray.open_dataset('./data/20190726_tb_latent_prev.nc').coords['location_id'].values
coord_dict = {'location_id':location_ids}
dataset = utils.saveDraws(beta_samples[0,:].reshape((1,-1)), u_samples, ['haq'], [['location_id']], 
                          ['pi_location'], coord_dict)

In [13]:
dataset

<xarray.Dataset>
Dimensions:      (cov: 1, draw: 10, location_id: 195)
Coordinates:
  * location_id  (location_id) int64 6 7 8 10 11 12 ... 351 376 385 422 435 522
  * draw         (draw) int64 1 2 3 4 5 6 7 8 9 10
  * cov          (cov) <U3 'haq'
Data variables:
    pi_location  (location_id, draw) float64 0.5525 0.5554 ... 1.463 1.466
    beta_global  (cov, draw) float64 -0.02594 -0.02608 ... -0.02608 -0.02601